# Bellingcat EDGAR Search Tool 🔎🏦

<a href="https://www.bellingcat.com"><img alt="Bellingcat logo: Discover Bellingcat" src="https://img.shields.io/badge/Discover%20Bellingcat-%20?style=for-the-badge&logo=data%3Aimage%2Fpng%3Bbase64%2CiVBORw0KGgoAAAANSUhEUgAAAA4AAAAYCAYAAADKx8xXAAABhGlDQ1BJQ0MgcHJvZmlsZQAAKJF9kT1Iw0AcxV9TS0UqDnZQEcxQneyiIo6likWwUNoKrTqYXPoFTRqSFBdHwbXg4Mdi1cHFWVcHV0EQ%2FABxdnBSdJES%2F5cUWsR4cNyPd%2Fced%2B8AoVllqtkTA1TNMtKJuJjLr4rBVwQwhhBEDEvM1JOZxSw8x9c9fHy9i%2FIs73N%2Fjn6lYDLAJxLHmG5YxBvEs5uWznmfOMzKkkJ8Tjxp0AWJH7kuu%2FzGueSwwDPDRjY9TxwmFktdLHcxKxsq8QxxRFE1yhdyLiuctzir1Tpr35O%2FMFTQVjJcpzmKBJaQRIo6klFHBVVYiNKqkWIiTftxD%2F%2BI40%2BRSyZXBYwcC6hBheT4wf%2Fgd7dmcXrKTQrFgcCLbX%2BMA8FdoNWw7e9j226dAP5n4Err%2BGtNYO6T9EZHixwBA9vAxXVHk%2FeAyx1g6EmXDMmR%2FDSFYhF4P6NvygODt0Dfmttbex%2BnD0CWulq%2BAQ4OgYkSZa97vLu3u7d%2Fz7T7%2BwHEU3LHAa%2FQ6gAAAAZiS0dEAAAAAAAA%2BUO7fwAAAAlwSFlzAAAuIwAALiMBeKU%2FdgAAAAd0SU1FB%2BgFHwwiMH4odB4AAAAZdEVYdENvbW1lbnQAQ3JlYXRlZCB3aXRoIEdJTVBXgQ4XAAAA50lEQVQ4y82SvWpCQRCFz25ERSJiCNqlUiS1b5AuEEiZIq1NOsGXCKms0wXSp9T6dskDiFikyiPc%2FrMZyf3FXSGQ0%2BzuzPl2ZoeVKgQ0gQ2wBVpVHlcDkjM5V%2FJ5nag6sJ%2FZX%2Bh%2FC7gEhqeAFKf7p1M9aB3b5oN1OomB7g1axUBPBr3GQHODHmOgqUF3MZAzKI2d4LWBV4H%2BMXDuJd1a7Cew1k7SwksaHC4LqNaw7aeX9GWHXkC1G1sTAS17Y3Kk2lnp4wNLiz0DrgLq8qt2MfmSSabAO%2FBBXp26dtrADPjOmN%2BAUdG7B3cE61l5hOZiAAAAAElFTkSuQmCC&logoColor=%23fff&color=%23000"></a><!--
--><a href="https://discord.gg/bellingcat"><img alt="Discord logo: Join our community" src="https://img.shields.io/badge/Join%20our%20community-%20?style=for-the-badge&logo=discord&logoColor=%23fff&color=%235865F2"></a><!--
--><a href="https://github.com/bellingcat/EDGAR"><img alt="Github logo: Explore the code" src="https://img.shields.io/badge/Explore%20the%20code-%20?style=for-the-badge&logo=github&color=%2329903b"></a>

A tool to search the US Securities and Exchange Commission EDGAR database of corporate filings and download the results as a spreadsheet.

The corporate filings held within the EDGAR database contain a wealth of quantitative and qualitative information on every legal entity that issues non-exempt securities in the United States.

This search tool was initially developed as part of the Bellingcat Tech Fellowship program, we hope it helps you utilise this incredible, free resource.

> <font color='#ffc107'>⚠️Warning:</font> Using both `search_keywords` and only `company_cik` at the same time might not work correctly, we recommend avoiding using both in a search (you can leave `search_keywords` blank and only specify `company_cik`)

In [ ]:
# @title Search EDGAR { display-mode: "form" }

# @markdown ### ⬅️ Click to search the EDGAR filings with the below parameters

search_keywords = "Volcano Safety" # @param {type:"string"}
start_date = '2024-01-01' # @param {type:"date"}
end_date = '2024-12-31' # @param {type:"date"}

TEXT_SEARCH_FILING_CATEGORIES = [
    None,
    "all_except_section_16",
    "all_annual_quarterly_and_current_reports",
    "all_section_16",
    "beneficial_ownership_reports",
    "exempt_offerings",
    "registration_statements",
    "filing_review_correspondence",
    "sec_orders_and_notices",
    "proxy_materials",
    "tender_offers_and_going_private_tx",
    "trust_indentures",
]

filing_type = "all" # @param ["all", "all_except_section_16", "all_annual_quarterly_and_current_reports", "all_section_16", "beneficial_ownership_reports", "exempt_offerings", "registration_statements", "filing_review_correspondence", "sec_orders_and_notices", "proxy_materials", "tender_offers_and_going_private_tx", "trust_indentures"]
company_cik = "" # @param {type:"string"}

# Handle empty arguments
search_keywords = "\"\"" if search_keywords == "" else search_keywords
filing_type = None if filing_type == "all" else filing_type
company_cik = None if company_cik == "" else company_cik

# Load Python dependencies
import pandas as pd
from datetime import datetime
from google.colab import data_table, files
from ipywidgets import widgets
from IPython.display import display
from IPython import get_ipython

data_table.enable_dataframe_formatter()

# Create the output files
output = f"edgar_search_results_{datetime.now().strftime('%d%m%Y_%H%M%S')}.csv"
logfile = f"./edgar_log_{datetime.now().strftime('%d%m%Y_%H%M%S')}.log"

# Enable Widgets
!jupyter nbextension enable --py widgetsnbextension >> {logfile} 2>&1

# Install the EDGAR search tool on the first run
![ ! -f "edgar_tool_installed" ] && echo -n "Loading the EDGAR Tool on first search..." && pip install edgar-tool >> {logfile} 2>&1 && pip install pandas==1.5.3 >> {logfile} 2>&1 && touch edgar_tool_installed && echo "Loaded."

# Run the tool with the query
!echo -n "Searching EDGAR..."
!edgar-tool text_search {search_keywords} --start_date {start_date} --end_date {end_date} --filing_type {filing_type} --entity_id {company_cik} --output {output} --browser firefox --min-wait 0.5 --max-wait 1.5 >> {logfile} 2>&1
exit_code = get_ipython().__dict__['user_ns']['_exit_code']
!echo "Done."

# Error handling
if exit_code == 2:
  print("\x1b[33m No results were found for your query.\x1b[0m")
elif exit_code != 0:
  print("\x1b[31m Something went wrong with the EDGAR tool, check your search and try again.\x1b[0m")
else:
  try:
    # Load results
    results = pd.read_csv(output)

    # Show download button
    btn = widgets.Button(description='Download Results')
    btn.on_click(lambda x: files.download(output))
    display(btn)

    # Display the results in a data table
    display(results)
  except FileNotFoundError as e:
    print("\x1b[31m Something went wrong with the EDGAR tool, please get in touch at contact-tech@bellingcat.com and help us improve the tool for everyone. \x1b[0m")

